# Structs

* A *struct* assembles several values of assorted types together into a single value, so you can deal with them as a unit. 
* Given a struct, you can read and modify its individual components. 
* A struct can have methods associated with it that operate on its components.

## Named-Field Structs

In [109]:
#[derive(Debug, Clone, PartialEq, Eq)]
pub struct Person {
    name: String,
    age: u8,
}

impl Person {
    pub fn new(name: &str, age: u8) -> Self {
        Person {
            name: String::from(name),
            age,
        }
    }

    pub fn name(&self) -> &str {
        &self.name
    }

    pub fn set_name(&mut self, name: &str) {
        self.name = String::from(name);
    }

    pub fn age(&self) -> u8 {
        self.age
    }

    pub fn set_age(&mut self, age: u8) {
        self.age = age;
    }

    pub fn greetings(&self) -> String {
        format!("Hello, my name is {} and I am {} years old.", self.name, self.age)
    }
}

In [110]:
fn greetings(person: &Person) -> String {
    format!("Hello, {}! You are {} years old.", person.name, person.age)
}

In [111]:
let name = String::from("Alice");

let mut person = Person {
    name,
    age: 30,
};

println!("{}", person.greetings());

let another_person = Person::new("Bob", 25);
println!("{}", another_person.greetings());

Hello, my name is Alice and I am 30 years old.
Hello, my name is Bob and I am 25 years old.


In [63]:
person.name()

"Alice"

In [64]:
person.set_age(31);
person.age()

31

In [52]:
println!("{person:?}");

Person { name: "Alice", age: 31 }


In [65]:
assert!(person == person.clone());

### Declaring a Struct

In [11]:
// A rectangle of eight-bit grayscale pixels.
struct GrayscaleMap {
    pixels: Vec<u8>,
    size: (usize, usize) // tuple (width, height)
}

### Creating a Struct Value

In [13]:
let width = 800;
let height = 600;

let size = (width, height);

let image = GrayscaleMap {
    pixels: vec![0; width * height],
    size
};

In [16]:
fn create_image(size: (usize, usize)) -> GrayscaleMap {
    let pixels = vec![0; size.0 * size.1];
    GrayscaleMap{pixels, size} // struct expression
}

create_image(size)

Error: `GrayscaleMap` doesn't implement `Debug`

### Accessing Struct Fields

In [5]:
let image = create_image((800, 600));

assert!(image.size == (800, 600));
assert!(image.pixels.len() == image.size.0 * image.size.1);

### Visibility of Struct

* Structs are private by default, visible only in the module where they’re declared. 
* You can make a struct visible outside its module by prefixing its definition with `pub`

In [17]:
// A rectangle of eight-bit grayscale pixels.
pub struct GrayscaleMap {
    pub pixels: Vec<u8>,
    pub size: (usize, usize)
}

## Tuple-Like Structs

* The second kind of struct type is called a tuple-like struct, because it resembles a tuple:

In [ ]:
#[derive(Debug, Clone, PartialEq, Eq)]
struct BoundingBox(usize, usize);

impl BoundingBox {
    fn width(&self) -> usize {
        self.0
    }

    fn height(&self) -> usize {
        self.1
    }
}

let image_bbox = BoundingBox(800, 600);

assert!(image_bbox.0 * image_bbox.1 == 480_000);
assert!(image_bbox.width() * image_bbox.height() == 480_000);


assert!(image_bbox == BoundingBox(800, 600));

* Individual elements of a tuple-like struct may be public or not:

In [19]:
pub struct BoundingBox(pub usize, pub usize);

### Tuple-Like Struct as Newtypes

* Tuple-like structs allow you to create types that behave like tuples but have a name and can implement traits
* This allows you to create newtypes for stricter type checking
* This is a workaround for the *orphan rule* - [New Type Pattern](https://doc.rust-lang.org/book/ch20-02-advanced-traits.html#using-the-newtype-pattern-to-implement-external-traits)

In [78]:
#[derive(Debug, Clone, Copy, PartialEq, PartialOrd)]
struct Distance(f64);

#[derive(Debug, Clone, Copy, PartialEq, PartialOrd)]
struct Velocity(f64);

#[derive(Debug, Clone, Copy, PartialEq, PartialOrd)]
struct Time(f64);

// A newtype for stricter type checking
fn calculate_distance(velocity: Velocity, time: Time) -> Distance {
    Distance(velocity.0 * time.0)
}

let velocity = Velocity(10.0);
assert!(velocity == Velocity(10.0));
assert!(velocity >= Velocity(9.0));

let time = Time(5.0);
let distance = calculate_distance(velocity, time);

println!("Distance traveled: {} kilometers", distance.0);

Distance traveled: 50 kilometers


## Unit-Like Structs

* The third kind of struct is a little obscure: it declares a struct type with no elements at all:

In [24]:
() // unit

()

In [27]:
struct Onesuch;

let o: Onesuch = Onesuch;

* This allows to create types that can act as tags or markers

## Defining Methods - `impl`

In [2]:
pub struct Queue {
    older: Vec<char>,
    younger: Vec<char>
}

impl Queue {
    pub fn push(&mut self, c: char) {
        self.younger.push(c);
    }

    pub fn pop(&mut self) -> Option<char> {
        if self.older.is_empty() {
            if self.younger.is_empty() {
                return None;
            }

            use std::mem::swap;
            swap(&mut self.older, &mut self.younger);
            self.older.reverse();
        }

        self.older.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.older.is_empty() && self.younger.is_empty()
    }
}

In [3]:
let mut q = Queue{ older: Vec::new(), younger: Vec::new() };

q.push('@');
q.push('$');
assert!(q.pop() == Some('@'));

q.push('*');
assert!(q.pop() == Some('$'));
assert!(q.pop() == Some('*'));
assert!(q.pop() == None);

assert!(q.is_empty());

In [4]:
impl Queue {
    pub fn split(self) -> (Vec<char>, Vec<char>) {
        (self.older, self.younger)
    }
}

In [5]:
let mut q = Queue { older: Vec::new(), younger: Vec::new() };
q.push('Q');
q.push('U');
assert!(q.pop() == Some('Q'));
q.push('X');

let (older, younger) = q.split();

assert_eq!(older, vec!['U']);
assert_eq!(younger, vec!['X']);

### `new` Method

* Constructor function - doesn't take `self` as an argument (**static function**)

In [15]:
impl Queue {
    pub fn new() -> Self {
        Queue { older: Vec::new(), younger: Vec::new() }
    }
}

In [24]:
let mut q = Queue::new();

q.push('a');
q.push('b');
q.push('c');

assert_eq!(q.pop(), Some('a'));
assert_eq!(q.pop(), Some('b'));
assert_eq!(q.pop(), Some('c'));
assert_eq!(q.pop(), None);

## Generic Structs

* Generics allow you to write flexible, reusable functions and types that can work with any data type. Instead of specifying concrete types, you use type parameters, which are placeholders for types that will be specified when the function or type is used.
* Generics use parameters passed in angle brackets to specify the types they work with. For example, `fn example<T>(param: T) {}` uses `T` as a generic type parameter.

In [30]:
pub struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    pub fn new() -> Stack<T> {
        Stack{ items: Vec::new() }
    }

    pub fn with_capacity(capacity: usize) -> Stack<T> {
        Stack{ items: Vec::with_capacity(capacity) }
    }

    pub fn with_items(items: &[T]) -> Stack<T> 
    where T: Clone
    {
        let mut stack = Stack::with_capacity(items.len());

        for item in items {
            stack.items.push(item.clone());
        }
        
        stack
    }

    pub fn push(&mut self, item: T) {
        self.items.push(item)
    }

    pub fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }

    pub fn is_empty(&self) -> bool {
        self.items.is_empty()
    }
}

* For static method calls, you can supply the type parameter explicitly using the turbofish `::<>` notation:

In [21]:
let mut s = Stack::<String>::new();

s.push(String::from("Hello"));
s.push(String::from("World"));
s.push("!!!".to_string());

* But in practice, Rust can figure it out for you:

In [23]:
let mut stack_1 = Stack::new();
let mut stack_2 = Stack::new();

stack_1.push("Hello");
stack_2.push(3.1415);

stack_1.push("World");
stack_2.push(2.71);

In [33]:
let mut stack_3 = Stack::with_items(&[1, 2, 3, 4, 5]);

## Deriving Common Traits for Structs

* Structs are not copyable or comparable by default
* You can derive the `Copy` and `Clone` traits for a struct type, but only if all its fields are copyable or clonable
* You can derive the `PartialEq` and `Eq` traits for a struct type, but only if all its fields are comparable for equality
* You can derive `Debug` for a struct type, but only if all its fields are printable

In [ ]:
#[derive(Copy, Clone, Debug, PartialEq, Default)]
#[repr(C)]
struct Point {
    x: f64,
    y: f64
}

In [58]:
let mut pt = Point{ x: 0.0, y: 1.0 };

let other_pt = pt.clone();
assert!(pt == other_pt);

pt.x = 42.42;
assert!(pt != other_pt);

println!("pt = {:?}", pt);
println!("other_pt = {:?}", other_pt);

pt = Point { x: 42.42, y: 1.0 }
other_pt = Point { x: 0.0, y: 1.0 }


In [114]:
let pt_default = Point::default();

assert!(pt_default == Point { x: 0.0, y: 0.0 });

In [117]:
#[derive(Copy, Clone, Debug, PartialEq, Default)]
struct Rectangle {
    top_left: Point,
    width: u32,
    height: u32,
}

In [89]:
let mut rect = Rectangle {
    height: 50,
    ..Default::default() // partial struct update syntax
};

println!("rect = {:?}", rect);

rect = Rectangle { top_left: Point { x: 0.0, y: 0.0 }, width: 0, height: 50 }


In [90]:
rect.height += 25;
rect.width = 100;
rect.top_left = Point { x: 10.0, y: 20.0 };

rect

Rectangle { top_left: Point { x: 10.0, y: 20.0 }, width: 100, height: 75 }

In [92]:
let new_rect = Rectangle {
    height: rect.height,
    ..rect // struct update syntax
};

# Builder Pattern

In [118]:
#[derive(Debug, Clone, PartialEq, Eq)]
pub struct Config {
    host: String,
    port: u16,
    use_ssl: bool,
}


struct ConfigBuilder {
    host: Option<String>,
    port: Option<u16>,
    use_ssl: Option<bool>,
}

impl ConfigBuilder {
    pub fn new() -> Self {
        ConfigBuilder {
            host: None,
            port: None,
            use_ssl: None,
        }
    }

    fn host(mut self, host: &str) -> Self {
        self.host = Some(host.to_string());
        self
    }

    fn port(mut self, port: u16) -> Self {
        self.port = Some(port);
        self
    }

    fn use_ssl(mut self, use_ssl: bool) -> Self {
        self.use_ssl = Some(use_ssl);
        self
    }

    fn build(self) -> Config {
        Config {
            host: self.host.unwrap_or("localhost".to_string()),
            port: self.port.unwrap_or(8080),
            use_ssl: self.use_ssl.unwrap_or(false),
        }
    }
}

In [100]:
let config_1 = ConfigBuilder::new()
    .host("example.com")
    .port(443)
    .use_ssl(true)
    .build();

config_1

Config { host: "example.com", port: 443, use_ssl: true }

In [101]:
let config_2 = ConfigBuilder::new()
    .host("example.org")
    .build();

config_2

Config { host: "example.org", port: 8080, use_ssl: false }

# PartialEq & Eq; PartialOrd & Ord

In [108]:
let NAN_value = f64::NAN;
let another_NAN = f64::NAN;

assert!(NAN_value.partial_cmp(&another_NAN) == None);
assert!(NAN_value != another_NAN);
assert!(NAN_value != std::f64::consts::PI);

In [126]:
let mut data = vec![std::f32::consts::PI, 1.0, 2.0, f32::NAN, -2.44, 4.0, 5.0];
//let mut data = vec![1, 3, 2, 5, 4, 665, 42];
println!("Before sorting: {:?}", data);

data.sort_by(|a, b| a.partial_cmp(b).unwrap_or(std::cmp::Ordering::Greater));
println!("After sorting: {:?}", data);

Before sorting: [3.1415927, 1.0, 2.0, NaN, -2.44, 4.0, 5.0]
After sorting: [1.0, 2.0, 3.1415927, NaN, -2.44, 4.0, 5.0]
